# Determinants de Slater

Les états à plusieurs particules de systèmes sans interaction peuvent être exprimer sous la forme de déterminant de Slater.

Un déterminant de Slater est un états à plusieurs particule qu'on peut construire simplement en appliquant des opérateur de création au vide. Par exemple,
$$ |\psi\rangle = c^\dagger_i c^\dagger_j |0 \rangle $$

est un déterminant de slatter à deux particules, une dans l'état étiquetté $i$ et l'autre dans l'état étiquetté $j$

Dans la notation de la deuxième quantification, un hamiltonien dont les états propres sont des déterminants de Slatter prennent la forme

$$ H = \sum_{ij} T_{ij} c^\dagger_i c_j $$

ou la matrice $T$ est hermitienne. Il s'agit de hamiltoniens *quadratiques*. Ce genre de hamiltonien sont ceux de système ayant seulement des contribution provenant de l'énergie cinétique et/ou de potentielles classique.

Le terme de saut du modèle de Hubbard (ainsi que le potentiel chimique), lorsqu'isolé tombe dans cette catégorie.

$$ H = -t \sum_{<ij>,\sigma }(c^\dagger_{i\sigma} c_{j\sigma} +c^\dagger_{j\sigma} c_{i\sigma})  - \mu \sum_{i\sigma} c^\dagger_{i\sigma}c_{i\sigma}+ U \sum_i c^\dagger_{i\uparrow}c^\dagger_{i\downarrow}c_{\downarrow}c_{i\uparrow}  $$


Le terme d'énergie cinétique du modèle de Hubbard est diagonnalisé par une transformé de Fourrier (en autant de dimensions que le modele en a). Dans le cas 1D:

$$ a_{k,\sigma} = \frac{1}{\sqrt{N}} \sum_j c_{j\sigma} e^{-ijk/N} $$
$$  -t \sum_{<ij>,\sigma }(c^\dagger_{i\sigma} c_{j\sigma} +c^\dagger_{j\sigma} c_{i\sigma}) = -2t \sum_{k\sigma} \mathrm{cos}(k) a^\dagger_{k\sigma}a_{k\sigma} $$

Les états propres à une particule de l'énergie cinétique sont $|k \sigma \rangle = a^\dagger_{k\sigma} |0 \rangle$ et ont l'énergie $-2t\mathrm{cos}(k)$

Les états propre à plusieurs particule sont peuvent être construit en effectuant appliquant plusieurs opérateur de création au vide, et l'énergie de l'état est simplement la somme des énergie de chacun des état à une particule occupé. À cause de la simplicité de la règle de composition, ce genre de hamiltionien (le cinétique) est completement résolu en solutionnant seulement le secteur à une particule. Ceci est faisable classiquement pour de très grand système, car la complexité algorithmique de ce problème est $O(N^3)$ dans le pire des cas.


Il existe des méthodes approximatives pour produire des solutions sous forme de determinant de slatter pour des Hamiltionien qui ne sont pas quadratique. La plus connue des ces méthode est certainenement la méthode Hartree-Fock. L'approximation Hartree-Fock est également classiquement tractable. 

### Et avec qiskit?

Ce qui suit est un exemple simple sur 4 qubits. Il s'agit du terme d'énergie cinétique pour un spin dans le modele de Hubbard 2x2.

In [ ]:
from qiskit import QuantumCircuit
from qiskit_nature.second_q.circuit.library import SlaterDeterminant,BogoliubovTransform
import numpy as np

#matrice de Sauts
T = np.array([
    [0,1,0,1],
    [1,0,1,0],
    [0,1,0,1],
    [1,0,1,0],
    ])

E,S = np.linalg.eigh(T) #calcule les valeurs propre et vecteurs propres

print(E)


La quantre état propres de la matrice T sont des état à une particule. L'état de moindre énergie à 2 particule est dégénéré 2 fois: le deuxième et le troisième état à une particule on une énergie nulle à précision machine (Et un calcule analytique montre qu'il doit s'agir d'exactement $0$). 

In [ ]:
deux_particules_A = SlaterDeterminant(S.T[[0,1]])#état 0 et état 1 occupé
deux_particules_B = SlaterDeterminant(S.T[[0,2]])#état 0 et état 2 occupé
deux_particules_A.draw('mpl')

In [ ]:
deux_particules_B.draw('mpl')

Si on veut préparer un état à 4 particules pour le modèle de Hubbard à 4 sites (avec le spin), on peut utiliser la somme directe des état propres trouvé précédement. Dans ce cas il y 4 déterminants de Slater avec la même énergie cinétique totale et 4 particules.

In [ ]:
S2 = np.zeros((8,8))
S2[:4,:4] = S2[4:,4:] = S 
#ordonne les spin par block: le block 4x4 en haut à gauche contient les terme pour 1 spin
#le block 4x4 en bas à droite pour l'autre.
quatre_particules_A = SlaterDeterminant(S2.T[[0,1,4,5]])
quatre_particules_B = SlaterDeterminant(S2.T[[0,2,4,5]])
quatre_particules_C = SlaterDeterminant(S2.T[[0,1,4,6]])
quatre_particules_D = SlaterDeterminant(S2.T[[0,2,4,6]])
quatre_particules_A.draw('mpl')

Ces circuit sont optimiser pour produire le déterminant de Slater demandé et aucun autre.
Qiskit nous permet aussi de contruit un circuit encodant la transformation linéaire entre les opérateur de création fermionique: `BogoliubovTransform`

Cette opérateur nous permet de construire des état un peu plus complexe comme des superposition de quelque déterminant de 

Le circuit suivant produit l'état A, encore.

On observe que les "optimisation" dans l'implémentation de `SlaterDeterminant` n'en sont pas lorsque qu'on encode dans l'ordre "spin majeur, site mineur"; la transformation de Bogoliubov est plus simple.

In [ ]:
Bogtrans = BogoliubovTransform(S2.T)
Bog_4_particules_A = QuantumCircuit(8)
Bog_4_particules_A.x(0)
Bog_4_particules_A.x(1)
Bog_4_particules_A.x(4)
Bog_4_particules_A.x(5)
Bog_4_particules_A.compose(Bogtrans,inplace=True)
Bog_4_particules_A.draw('mpl')

La transformation nous offre la possibilité de faire des superposition de ces déterminant. 
Selon l'article d'Alexandre Choquette, la combinaison optimal est $(|A\rangle - |C\rangle)/\sqrt(2)$



In [ ]:
Optimal = QuantumCircuit(8)
Optimal.x(0)
Optimal.x(4)
Optimal.h(1)
Optimal.cx(1,3)
Optimal.cx(3,5)
Optimal.cx(5,6)
Optimal.x(3)
Optimal.z(3)
Optimal.x(6)
Optimal.compose(Bogtrans,inplace=True)
Optimal.draw('mpl')

In [ ]:
from qiskit import qpy

with open('trans_bogoliubov.qpy','wb') as slt:
    qpy.dump(Bogtrans,slt)

with open('4part_slater_bogo.qpy','wb') as slt:
    qpy.dump(Bog_4_particules_A,slt)

with open('2slaters_opt_bogo.qpy','wb') as slt:
    qpy.dump(Optimal,slt)

In [ ]:
from qiskit import qpy

with open('2slaters_opt_bogo.qpy', 'rb') as fd:
    new_qc = qpy.load(fd)[0]

In [ ]:
new_qc.draw('mpl')

In [ ]:
with open('trans_bogoliubov.qasm','w') as slt:
    slt.write(Bogtrans.qasm())

with open('4part_slater_bogo.qasm','w') as slt:
    slt.write(Bog_4_particules_A.qasm())

with open('2slaters_opt_bogo.qasm','w') as slt:
    slt.write(Optimal.qasm())

In [ ]:

opt = QuantumCircuit.from_qasm_file('2slaters_opt_bogo.qasm')
opt.draw('mpl')

Pour le modèle de Hubbard 1x4.

In [ ]:

#matrice de Sauts
T = np.array([
    [0,1,0,0],
    [1,0,1,0],
    [0,1,0,1],
    [0,0,1,0],
    ])

E,S = np.linalg.eigh(T) #calcule les valeurs propre et vecteurs propres
print("Énergies particules libre:", E)

In [ ]:
S2 = np.zeros((8,8))
S2[:4,:4] = S2[4:,4:] = S 
Slater_lin_V1 = SlaterDeterminant(S2.T[[0,1,4,5]])#état 0 et état 1 occupé
Slater_lin_V1.draw('mpl')

In [ ]:
bogtrans = BogoliubovTransform(S2.T)
Slater_lin_V2 = QuantumCircuit(8)
Slater_lin_V2.x(0)
Slater_lin_V2.x(1)
Slater_lin_V2.x(4)
Slater_lin_V2.x(5)
Slater_lin_V2.compose(Bogtrans,inplace=True)
Slater_lin_V2.draw('mpl')